# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115195e+02     1.561257e+00
 * time: 0.748162031173706
     1     9.965620e+00     9.522462e-01
 * time: 3.1196770668029785
     2    -1.180812e+01     9.838244e-01
 * time: 3.7786219120025635
     3    -3.394501e+01     7.378069e-01
 * time: 4.787785053253174
     4    -4.702884e+01     5.986818e-01
 * time: 5.753901958465576
     5    -5.684669e+01     2.470319e-01
 * time: 6.731514930725098
     6    -5.954786e+01     2.670282e-01
 * time: 7.390548944473267
     7    -6.070149e+01     1.039068e-01
 * time: 8.063516855239868
     8    -6.122440e+01     5.455785e-02
 * time: 8.714869022369385
     9    -6.151579e+01     3.801283e-02
 * time: 9.365562915802002
    10    -6.177166e+01     3.827812e-02
 * time: 10.01638388633728
    11    -6.192572e+01     2.288692e-02
 * time: 10.665322065353394
    12    -6.204256e+01     1.756147e-02
 * time: 11.327800989151001
    13    -6.208817e+01     1.439013e-02
 * time: 11.975620985031128
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557717
    AtomicNonlocal      14.8522672
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485384 
    Xc                  -19.3336825

    total               -62.261666461424
